<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/geemap/tree/master/examples/template/template.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/template/template.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/template/template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Clasificación supervisada SVM
Script trabajado por Francisco Albornoz (2021-04-28 12:51:11 ) / Última modificación / Análisis y clasificación supervisada región de Aysen

<img src=https://www.ecopaysen.cl/wp-content/uploads/2019/12/ecopaysen_marca_web.png>

In [ ]:
# Installs geemap package
# import subprocess

# try:
#     import geemap
# except ImportError:
#     print("Installing geemap ...")
#     subprocess.check_call(["python", "-m", "pip", "install", "geemap"])
# !pip install scikit_learn


In [1]:
import os
import ee
import geemap
import xarray_leaflet
import rasterio 
from rasterio import plot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import pandas as pd

## Crear un mapa interactivo 
The default basemap is `Google Maps`. [Additional basemaps](https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py) can be added using the `Map.add_basemap()` function. 

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## Escogemos la imagen landsat 

In [3]:
centroid = ee.Geometry.Point([-72, -44])

landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(centroid) \
    .filterDate('2020-01-01', '2021-03-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

landsat_vis = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(centroid, 8)
Map.addLayer(landsat, landsat_vis, "Landsat-8")


## Importar rasters locales 

In [4]:
%pwd

'/mnt/c/users/usuario/Documents/GitHub/VitoAlbornoz/GEE'

In [5]:
# verificar en el sistema del pc cual es la ruta
# os.path.expanduser('/mnt/c/Users/R7 3700X/Documents/GitHub/VitoAlbornoz/') 
# os.path.expanduser('/mnt/c/Users/Usuario/Documents/Github/Vitoalbornoz/')

In [6]:
# verificar en el sistema del pc cual es la ruta
out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/'), 'Data')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

raster_aysen = os.path.join(out_dir, 'landcover_2018_aysen.tif')

In [7]:
out_dir

'/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data'

In [8]:
raster_aysen

'/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data/landcover_2018_aysen.tif'

In [9]:
Map.add_raster(raster_aysen, colormap= 'terrain', layer_name='Landcover 2018')

In [10]:
landcover = ee.Image("users/franciscoalbornoz/landcover_sur")
landcover

In [11]:
raster = rasterio.open(out_dir+'/landcover_2018_aysen.tif')

In [12]:
raster.bounds

BoundingBox(left=-73.169644655, bottom=-46.394042378, right=-70.719691266, top=-43.705745621)

In [13]:
raster.crs

CRS.from_epsg(4326)

In [14]:
raster

<open DatasetReader name='/mnt/c/Users/Usuario/Documents/GitHub/VitoAlbornoz/Data/landcover_2018_aysen.tif' mode='r'>

In [15]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")

landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")

Map

Map(center=[-45.0498939995, -71.9446679605], controls=(WidgetControl(options=['position'], widget=HBox(childre…

## Obteniendo los valores para el Training Datasets

In [19]:
work_dir = os.path.expanduser('../Data/shapes/')

merged_shp = os.path.join(work_dir, 'roi_h.shp')
val_shp= os.path.join(work_dir, 'puntos_validacion.shp')

In [20]:
merged = geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [21]:
val = geemap.shp_to_ee(val_shp)
Map.addLayer(val, {}, 'Áreas de entrenamiento2')

'utf-8' codec can't decode byte 0xf1 in position 0: invalid continuation byte


AttributeError: 

The image argument in 'addLayer' function must be an instace of one of ee.Image, ee.Geometry, ee.Feature or ee.FeatureCollection.

In [ ]:
# print(val_shp)

In [ ]:
dataset = mosaico.sampleRegions(**{
    'collection': merged,
    'properties': ['IDH'],
    'scale' : 30 ,
})

In [ ]:
datasetval = mosaico.sampleRegions(**{
    'collection': val,
    'properties': ['IDH'],
    'scale': 30,
})

In [ ]:
print(dataset.size().getInfo())

In [ ]:
print(datasetval.size().getInfo())

In [ ]:
dataset.propertyNames()

In [ ]:
datasetval.propertyNames()

In [ ]:
print(dataset.first().getInfo())

In [ ]:
print(datasetval.first().getInfo())

In [ ]:
out_csv = os.path.join(out_dir, 'Dataset_1.csv')
# geemap.extract_values_to_points(merged, mosaico, out_csv)

In [ ]:
out_csv2 =os.path.join(out_dir, 'Dataset_2.csv')
# geemap.extract_values_to_points(validacion, mosaico, out_csv)

In [ ]:
df = pd.read_csv(out_csv)

In [ ]:
df

In [ ]:
df2 = pd.read_csv(out_csv2)

In [ ]:
df2

### Dividiendo en muestras y datos de validación

In [ ]:
# Guardamos las bandas
bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

# Esta propiedad guarda las clases de landcover
label = 'IDH'

# Overlay the points on the imagery to get training.
# sample = landsat.select(bands).sampleRegions(**{
#   'collection': points,
#   'properties': [label],
#   'scale': 30
# })

# Adds a column of deterministic pseudorandom numbers. 
# sample = dataset.randomColumn()

# split = 0.7 

# training = sample.filter(ee.Filter.lt('random', split))
# validation = sample.filter(ee.Filter.gte('random', split))

In [ ]:
# training.first().getInfo()

In [ ]:
# validation.first().getInfo()

## Entrenando el modelo 

In [ ]:
from sklearn import svm
from sklearn import ensemble

In [ ]:
feature_names = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']
label = "IDH"

In [ ]:
# X = df[feature_names]
# y= df[label]

In [ ]:
classifi = ee.Classifier.libsvm('Voting','C_SVC','RBF', gamma = 0.5 , cost = 10 )

In [ ]:
# clf =svm.SVC()
# clf.fit(X, y)

In [ ]:
trained = classifi.train(dataset, label, bands)
classified = mosaico.classify(trained)

In [ ]:
trained

In [ ]:
classified

### Clasificando la imagen

In [ ]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [ ]:
land = classified.set('classification_class_values', class_values)
landcoverf = land.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcoverf, {}, 'Land cover SVM')
Map

### Categorizando según las clases 

In [ ]:
# landcover.getInfo()

In [ ]:
# class_values = [1,
#  2,
#  3,
#  4,
#  5,
#  6,
#  7,
#  8,
#  9,
#  10,
#  11,
#  12,
#  13,
#  14,
#  15,
#  16]

In [ ]:
# class_palette = ['476ba1',
#  'd1defa',
#  'decaca',
#  'd99482',
#  'ee0000',
#  'ab0000',
#  'b3aea3',
#  '68ab63',
#  '1c6330',
#  'b5ca8f',
#  'a68c30',
#  'ccba7d',
#  'e3e3c2',
#  'caca78',
#  '99c247',
#  '78ae94']


In [ ]:
# landcoverf = result.set('classification_class_values', class_values)
# landcoverf = landcoverf.set('classification_class_palette', class_palette)

In [ ]:
# Map.addLayer(landcoverf, {}, 'Land cover clasificado' )
# Map

### Visualizando el resultado


In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))


In [ ]:
Map

### Agregando título al mapa

In [ ]:
# Map.add_legend('Clasificación supervisada CART')
# Map

### Evaluación de presición / Accuracy assessment

In [ ]:
train_accuracy = trained.confusionMatrix()

In [ ]:
train_accuracy.getInfo()

In [ ]:
train_accuracy.accuracy().getInfo()

In [ ]:
train_accuracy.kappa().getInfo()

In [ ]:
train_accuracy.producersAccuracy().getInfo()

In [ ]:
train_accuracy.consumersAccuracy().getInfo()

#### Set de datos de validación

In [ ]:
validated = datasetval.classify(trained)

In [ ]:
validated.first().getInfo()

In [ ]:
test_accuracy = validated.errorMatrix('IDH', 'classification')

In [ ]:
test_accuracy.getInfo()

In [ ]:
test_accuracy.accuracy().getInfo()

In [ ]:
test_accuracy.kappa().getInfo()

In [ ]:
test_accuracy.producersAccuracy().getInfo()

In [ ]:
test_accuracy.consumersAccuracy().getInfo()

### Descargando la matrix

In [ ]:
# import csv
# import os

# out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/R7 3700X/'), 'Desktop')
# training_csv = os.path.join(out_dir, 'train_accuracy.csv')
# testing_csv = os.path.join(out_dir, 'test_accuracy.csv')

# with open(training_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(train_accuracy.getInfo())
    
# with open(testing_csv, "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerows(test_accuracy.getInfo())

### Exportando 

In [ ]:
# import os
# out_dir = os.path.join(os.path.expanduser('/mnt/c/Users/R7 3700X/'), 'Desktop')
# out_file = os.path.join(out_dir, 'landcover.tif')

In [ ]:
# geemap.ee_export_image(landcover, filename=out_file, scale=900)


In [ ]:
# geemap.ee_export_image_to_drive(landcover, description='landcover', folder='export', scale=900)